In [0]:
# Define your Volume path
volume_path = "/Volumes/vstone_project/db_project/raw_data/"

# 1. Read the main file
df_main = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"{volume_path}1_main.csv")

# 2. Split the data into 4 equal parts (approx 25% each)
chunks = df_main.randomSplit([0.25, 0.25, 0.25, 0.25], seed=42)

chunk1, chunk2, chunk3, chunk4 = chunks[0], chunks[1], chunks[2], chunks[3]

# 3. Save Chunks back to the Volume
# Chunk 1 & 2 stay as CSV
chunk1.write.format("csv").option("header", "true").mode("overwrite").save(f"{volume_path}chunk1_initial")
chunk2.write.format("csv").option("header", "true").mode("overwrite").save(f"{volume_path}chunk2_incremental")

# Chunk 3 to JSON
chunk3.write.format("json").mode("overwrite").save(f"{volume_path}chunk3_json")

# Chunk 4 to XML (Requires a specific spark provider)
chunk4.write.format("xml").option("rootTag", "records").option("rowTag", "item").mode("overwrite").save(f"{volume_path}chunk4_xml")

# 4. Handle other files (Moving them to specific folders if needed)
# For catalogs and final_geographic, you can leave them in the root or move them.
print("Data Chunking Complete.")